# separatio based on date coloumn

In [0]:

from pyspark.sql.functions import col, date_format

df = spark.table("my_unity_catalog_metastore.default.sample_people")
# df.show()

#distinct dates from birthd column
unique_dates = df.select("birthd").distinct().collect()

count = 1

# unique date and save separate table
for row in unique_dates:
    
    date_str = row['birthd'].strftime('%Y_%m_%d')  # e.g., 1995_08_15
    table_name = f"my_unity_catalog_metastore.default.{count}_sep_table{date_str}"
    
    # Filter data for that date
    filtered_df = df.filter(col("birthd") == row["birthd"])

    # Save as a new Delta table in Unity Catalog
    filtered_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

    print(f"Save table: {table_name}")
    count += 1

In [0]:

from pyspark.sql.functions import col, date_format
from datetime import datetime

df = spark.table("my_unity_catalog_metastore.sample_people.row1_dataset")

# df.show()

# distinct dates from birthd column
unique_dates = df.select("date").distinct().collect()

count = 1
# unique date and save separate table
for row in unique_dates:

    date_str = row['date'].strftime('%Y_%m_%d')  # e.g., 1995_08_15
    table_name = f"my_unity_catalog_metastore.sample_people.table{count}_{date_str}"
    
    # # Filter data for that date
    filtered_df = df.filter(col("date") == row["date"])

    # Save as a new Delta table in Unity Catalog
    filtered_df.write.format("delta").mode("overwrite").saveAsTable(table_name)

    print(f"Save table: {table_name}")
    count += 1

## parallel execution 

In [0]:
from pyspark.sql.functions import col
from concurrent.futures import ThreadPoolExecutor

# Load main table
df = spark.table("my_unity_catalog_metastore.sample_people.row1_dataset")

# Get unique dates
unique_dates = df.select("date").distinct().collect()

# Function to save one date partition as a table
def save_partition(row, count):
    date_val = row["date"]
    date_str = date_val.strftime('%Y_%m_%d')
    table_name = f"my_unity_catalog_metastore.sample_people.table{count}_{date_str}"
    df.filter(col("date") == date_val) \
      .write.format("delta").mode("overwrite") \
      .saveAsTable(table_name)
    print(f"Saved table: {table_name}")

# Run in parallel
with ThreadPoolExecutor() as executor:
    for i, row in enumerate(unique_dates, start=1):
        executor.submit(save_partition, row, i)
